intalace knihoven

In [25]:
%pip install aiohttp
%pip install asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Importování knihoven

In [26]:
import aiohttp
import asyncio

Získání přihlašovacích údajů

In [27]:
async def getToken(username, password):
    keyurl = "http://127.0.0.1:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            # print(resp.status)
            keyJson = await resp.json()
            # print(keyJson)

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            # print(resp.status)
            tokenJson = await resp.json()
            # print(tokenJson)
    return tokenJson.get("token", None)
            


Funkce pro query

In [28]:
def query(q, token):
    async def post(variables):
        gqlurl = "http://127.0.0.1:33001/api/gql"
        payload = {"query": q, "variables": variables}
        # headers = {"Authorization": f"Bearer {token}"}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            # print(headers, cookies)
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                # print(resp.status)
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post 


Nějaký kód od hrbloka

In [29]:
from itertools import product
from functools import reduce

def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            # print("flatList", row)
            yield row
            
def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    # print("flatDict.result", result)
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced
        
def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"


Definice
    - přihlašovacích údajů
    - query pro získání dat
    - mapování dat

In [30]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"

queryStr = """
{
  result: eventPage(
    where: {
      _and: [
        {type_id: {_in: [
          "b87d7be6-8fd4-11ed-a6d4-0242ac110002",
          "b87d3ff0-8fd4-11ed-a6d4-0242ac110002"]}},
        {startdate: {_gt: "2022-09-01T00:00:00"}},
        {enddate: {_lt: "2023-09-01T00:00:00"}}
      ]
    }
  ) {
    id
    name
    startdate
    enddate
    eventType {
      id
      name
    }
    groups {
      id
      name
    }
  }
}
"""

mappers = {
    "id": "id",
    "name": "name",
    "startdate": "startdate",
    "enddate": "enddate",
    "eventTypeId": "eventType.id",
    "eventTypeName": "eventType.name",
    # "eventduration": "presences.event.duration",
    "groupsId": "groups.id",
    "groupsId": "groups.name"
}

async def fullPipe():
    global pandasData
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})
    
    data = response.get("data", None)
    result = data.get("result", None)
    # response = await qfunc({})
    flatData = flatten(result, {}, mappers)
    print(list(flatData))
    
    #print(response)
# async def main():
#     flatData = await fullPipe()
#     with open(result.json, w, encodin= utf-)
    #

    #
   # pandasData = toTable(flatData)
   # return pandasData
  

await fullPipe()
# print(list(flatData))


[{'id': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'name': 'Zkouška', 'startdate': '2023-04-19T08:00:00', 'enddate': '2023-04-19T09:00:00', 'eventTypeId': 'b87d3ff0-8fd4-11ed-a6d4-0242ac110002', 'eventTypeName': 'Ostatní', 'groupsId': '22-5KB-A'}, {'id': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'name': 'Zkouška', 'startdate': '2023-04-19T08:00:00', 'enddate': '2023-04-19T09:00:00', 'eventTypeId': 'b87d3ff0-8fd4-11ed-a6d4-0242ac110002', 'eventTypeName': 'Ostatní', 'groupsId': '22-5KB-B'}, {'id': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'name': 'Zkouška', 'startdate': '2023-04-19T08:00:00', 'enddate': '2023-04-19T09:00:00', 'eventTypeId': 'b87d3ff0-8fd4-11ed-a6d4-0242ac110002', 'eventTypeName': 'Ostatní', 'groupsId': '22-5KB-C'}, {'id': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'name': 'Zkouška', 'startdate': '2023-04-19T08:00:00', 'enddate': '2023-04-19T09:00:00', 'eventTypeId': 'b87d3ff0-8fd4-11ed-a6d4-0242ac110002', 'eventTypeName': 'Ostatní', 'groupsId': '22-5KB-D'}, {'id': '45b2df80-ae0f-1

Transforamce dat

import json
from datetime import datetime
import copy
import pandas as pd
from collections import defaultdict

def transform_events(data):
    transformed_events = []
    
    for event in data["data"]["eventPage"]:
        event_type = event.pop("eventType")  # Odstranění původního eventType
        event["eventtype_id"] = event_type["id"]  # Přidání nového klíče eventtype_id
        event["eventtype_name"] = event_type["name"]  # Přidání nového klíče eventtype_name

        start_date = datetime.fromisoformat(event["startdate"])
        end_date = datetime.fromisoformat(event["enddate"])
        duration = end_date - start_date
        event["duration"] = duration.total_seconds() / 60  # Převod na minuty

        # Odstranění původních atributů "startdate" a "enddate"
        del event["startdate"]
        del event["enddate"]

        for group in event["groups"]:
            event_copy = copy.deepcopy(event)
            event_copy["group_id"] = group["id"]
            event_copy["group_name"] = group["name"]
            transformed_events.append(event_copy)

    for event in transformed_events:
        if "groups" in event:
            del event["groups"]

    return transformed_events

def aggregate_events(events):
    aggregated_data = defaultdict(lambda: defaultdict(lambda: {"total_duration": 0.0}))

    for event in events:
        group_name = event["group_name"]
        name = event["eventtype_name"]
        duration = event["duration"]
        
        aggregated_data[group_name][name]["total_duration"] += duration
    
    # Konverze na list dict pro výstup
    output_data = []
    for group_name, events in aggregated_data.items():
        for name, details in events.items():
            output_data.append({
                "group_name": group_name,
                "eventtype_name": name,
                "total_duration": details["total_duration"]
            })
    
    return output_data

f = "data.json"
with open(f, 'r', encoding='utf-8') as file:
    data_json = json.load(file)


transformed_data = transform_events(data_json)
aggregated_data = aggregate_events(transformed_data)

# ====== transformovana data do JSONu a excelu ==========
# with open('transformed.json', 'w', encoding='utf-8') as outfile:
#     json.dump(transformed_data, outfile, indent=4, ensure_ascii=False)

# df = pd.DataFrame(transformed_data)
# df.to_excel('tranformed.xlsx', index=False)

# ====== agregovana data do JSONu a excelu ==============
with open('aggregated.json', 'w', encoding='utf-8') as outfile:
    json.dump(aggregated_data, outfile, indent=4, ensure_ascii=False)

df = pd.DataFrame(aggregated_data)
df.to_excel('aggregated.xlsx', index=False)